<a href="https://colab.research.google.com/github/respect5716/deep-learning-paper-implementation/blob/main/07_Anomaly/DevNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DevNet

## 0. Paper

### Info
* Title: Deep Anomaly Detection with Deviation Networks
* Author: Guansong Pang
* Task: Anomaly Detection
* URL: https://arxiv.org/abs/1911.08623


### Features
* Dataset: Credit Card Fraud Detection, [link](https://www.kaggle.com/mlg-ulb/creditcardfraud)

### Reference
* https://github.com/GuansongPang/deviation-network
* https://github.com/Choubo/deviation-network-image

## 1. Setting

In [ ]:
!pip install torch==1.9.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install -q wandb
!pip install -q transformers
!pip install -q pytorch_lightning

In [2]:
import os
import wandb
from glob import glob
from typing import Optional

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

import pytorch_lightning as pl

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

_ = pl.utilities.seed.seed_everything(999)

Global seed set to 999


In [3]:
import os
if os.path.isdir('ds_utils'):
    !rm -rf ds_utils
!git clone -q https://github.com/respect5716/ds_utils.git

from ds_utils.pytorch_lightning import BaseModule, BaseDataModule, CheckpointCallback, AttributeDict
from ds_utils.metric import accuracy

In [4]:
args = AttributeDict({
    'batch_size': 256,
    'base_dir': '/content/drive/Shared drives/Yoon/Project/Doing/Deep Learning Paper Implementation',

    'trainer': {
        # 'accelerator': 'dp',
        'gpus': -1,
        'log_every_n_steps': 1,
        'num_sanity_val_steps': -1,
        'max_epochs': 1,
        # 'check_val_every_n_epoch': 1,
        'val_check_interval': 100,

    },

    'optimizer': {
        'name': 'adam',
        'params': {
            'lr': 1e-3,
            'weight_decay': 5e-4
        }
    },

    'scheduler': {
        'name': 'linear',
        'warmup_size': 0
    }
})

## 2. Data

### Prepare

In [ ]:
data_path = os.path.join(args.base_dir, 'data', 'credit_card_fraud_detection.zip')
!unzip -q "{data_path}" -d data

In [6]:
data = pd.read_csv('data/data.csv')
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [7]:
data['Amount'] /= data['Amount'].max()
data['Time'] /= data['Time'].max()

In [8]:
X = data.drop('Class', axis=1)
y = data['Class']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((227845, 30), (56962, 30), (227845,), (56962,))

### Dataset & DataLoader

In [10]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.inliers = list(y.loc[y == 0].index)
        self.outliers = list(y.loc[y == 1].index)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        label = idx % 2 # (0: normal, 1: anomaly)
        item_idx = np.random.choice(self.outliers) if label else np.random.choice(self.inliers)
        item = torch.tensor(self.X.loc[item_idx], dtype=torch.float32)
        return item, label

In [11]:
train_dataset = Dataset(X_train, y_train)
test_dataset = Dataset(X_test, y_test)

train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=args.batch_size, num_workers=12)
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False, batch_size=args.batch_size, num_workers=12)

In [12]:
inputs = next(iter(train_loader))
inputs[0].size(), inputs[1].size()

(torch.Size([256, 30]), torch.Size([256]))

## 3. Model

In [13]:
class DeviationLoss(nn.Module):
    def __init__(self, confidence_margin=5., num_ref=5000):
        super().__init__()
        self.confidence_margin = confidence_margin
        self.num_ref = num_ref

    def forward(self, pred, true):
        ref = torch.normal(mean=0., std=1., size=(self.num_ref,), device=pred.device)
        dev = (pred - ref.mean()) / ref.std()
        inlier_loss = torch.abs(dev)
        outlier_loss = (self.confidence_margin - dev).clamp(min=0.)
        loss = torch.mean((1 - true) * inlier_loss + true * outlier_loss)
        return loss

class Model(BaseModule):
    def __init__(self, **hparams):
        super().__init__()
        self.save_hyperparameters(hparams)

        self.fc1 = nn.Linear(self.hparams.input_dim, 20)
        self.fc2 = nn.Linear(20, 1)
        self.criterion = DeviationLoss()
        
    def forward(self, x):
        return self.fc2(F.relu(self.fc1(x))).squeeze()
    
    def step(self, batch, mode):
        inputs, labels = batch
        preds = self(inputs)
        loss = self.criterion(preds, labels)
        roc_auc = roc_auc_score(labels.cpu().numpy(), preds.data.cpu().numpy())
        log = {f'{mode}/loss': loss, f'{mode}/roc_auc': roc_auc}
        self.log_dict(log, prog_bar = True, on_step = True, on_epoch = True)
        return loss

In [14]:
model = Model(input_dim=30, **args)

## 4. Experiment

### Train

In [15]:
logger = pl.loggers.WandbLogger(
    project = 'paper',
    log_model = False,
    reinit = True
)

logger.watch(model)

wandb: Currently logged in as: respect5716 (use `wandb login --relogin` to force relogin)


In [16]:
lr_monitor = pl.callbacks.LearningRateMonitor()
ckpt_callback = CheckpointCallback('ckpt', metric='valid/roc_auc_epoch', mode='max')

In [17]:
trainer = pl.Trainer(
    logger = logger,
    callbacks = [lr_monitor, ckpt_callback],
    **args.trainer
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [18]:
trainer.fit(model, train_loader, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type          | Params
--------------------------------------------
0 | fc1       | Linear        | 620   
1 | fc2       | Linear        | 21    
2 | criterion | DeviationLoss | 0     
--------------------------------------------
641       Trainable params
0         Non-trainable params
641       Total params
0.003     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 999


STEP: 000000 | valid/roc_auc_epoch: 0.358 | model saved (-10000000000.000 -> 0.358)


Training: -1it [00:00, ?it/s]

...

STEP: 000599 | valid/roc_auc_epoch: 0.986 | model not saved (best metric: 0.986)


Validating: 0it [00:00, ?it/s]

STEP: 000699 | valid/roc_auc_epoch: 0.978 | model not saved (best metric: 0.986)


Validating: 0it [00:00, ?it/s]

STEP: 000799 | valid/roc_auc_epoch: 0.977 | model not saved (best metric: 0.986)


### Test

In [19]:
model = Model.load_from_checkpoint('ckpt').freeze()

In [21]:
res = trainer.test(model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test/loss': 0.32765042781829834,
 'test/loss_epoch': 0.32765042781829834,
 'test/roc_auc': 0.9777730703353882,
 'test/roc_auc_epoch': 0.9777730703353882}
--------------------------------------------------------------------------------
